In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
!nvidia-smi

Tue May  7 13:24:49 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.40.04    Driver Version: 418.40.04    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00005E6A:00:00.0 Off |                    0 |
| N/A   46C    P0    57W / 149W |      0MiB / 11441MiB |      1%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CUDNN_USE_AUTOTUNE'] = '0'

In [5]:
import tensorflow as tf
config = tf.ConfigProto(intra_op_parallelism_threads=0, 
                        inter_op_parallelism_threads=0, 
                        allow_soft_placement=True)
session = tf.Session(config=config)
tf.keras.backend.set_session(session)

In [6]:
import random
import numpy as np

seed = 11

np.random.seed(seed)
random.seed(seed)
tf.random.set_random_seed(seed)

In [15]:
from fewshot.data_provider.dataset import Dataset
from fewshot.backbones import ConvNet

from fewshot.algorithms.backbone_train import simple_cosine_layer_cross_entropy_train
from fewshot.algorithms.fewshot_test import baseline_fewshot_test
from fewshot.algorithms.fewshot_models import BaselineFewShotModel

import tqdm
from scipy import stats

## Initialize

Data parameters:

In [16]:
img_width = 84
img_height = 84
img_depth = 3

In [17]:
dataset_root = "../../../data/"
dataset_name = 'cub'
csv_name = "data_84x84.csv"

base_num_classes = 100
novel_num_classes = 50
val_num_classes = 50  # not used in baseline algo

Backbone training parameters:

In [18]:
# backbone training
backbone_training_batch_size = 16
backbone_training_epochs = 20

backbone_generator_args={
    "flip_prob": 0.5,
    "color_jitter_prob": 1.0,
    "hue_range": (0.6, 1.4),
    "saturation_range": (0.6, 1.4),
    "value_range": (0.6, 1.4)
}

Few-shot training parameters:

In [19]:
# fewshot training and testing
n_way = 5
k_shot = 5
fewshot_batch_size = 4
support_epochs = 100
query_size = 16

support_generator_args={
    "flip_prob": 0.5,
    "color_jitter_prob": 1.0,
    "hue_range": (0.6, 1.4),
    "saturation_range": (0.6, 1.4),
    "value_range": (0.6, 1.4)
}

n_episodes = 10

In [20]:
checkpoint_dir = '../../../data/few-models/fewshot/checkpoints'
log_dir = '../../../data/few-models/fewshot/logs'

## Data

Create dataset:

In [21]:
dataset_dir = os.path.join(dataset_root, dataset_name)
dataset = Dataset(dataset_dir=dataset_dir, 
                  csv_name=csv_name, 
                  image_size=(img_width, img_height))

Create backbone dataset:

In [22]:
backbone_dataset, val_fewshot_dataset = dataset.split_by_classes(train_size=base_num_classes,
                                                                 random_state=seed)

Split by classes with train size = 100 (seed = 11)
Train classes: 100
Test classes: 100
Train data: 3043 samples
Test data:  2990 samples


/mnt/disk/venv/dpy3/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Create few-shot dataset:

In [23]:
val_dataset, fewshot_dataset = val_fewshot_dataset.split_by_classes(train_size=val_num_classes,
                                                                    random_state=seed)

Split by classes with train size = 50 (seed = 11)
Train classes: 50
Test classes: 50
Train data: 1510 samples
Test data:  1480 samples


## Backbone

In [24]:
model_name = 'baseline'

Create model:

In [25]:
backbone = ConvNet(input_size=(img_width, img_height, img_depth))
backbone.set_trainable(True)

Instructions for updating:
Colocations handled automatically by placer.


Train model:

In [26]:
backbone_optimizer = tf.keras.optimizers.Adam(lr=1e-3)

In [27]:
simple_cosine_layer_cross_entropy_train(
    backbone,
    backbone_dataset.get_batch_generator(batch_size=backbone_training_batch_size,
                                         shuffle=True,
                                         generator_args=backbone_generator_args),
    epochs=backbone_training_epochs,
    optimizer=backbone_optimizer,
    model_name=model_name,
    checkpoint_dir=checkpoint_dir,
    period=10,
    tensorboard=True,
    log_dir=log_dir,
    use_multiprocessing=True,
    workers=4,
    verbose=2
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
 - 7s - loss: 4.5677 - acc: 0.0194
Epoch 2/20
 - 6s - loss: 4.4756 - acc: 0.0332
Epoch 3/20
 - 6s - loss: 4.4160 - acc: 0.0483
Epoch 3/20
Epoch 4/20
Epoch 4/20
 - 6s - loss: 4.3874 - acc: 0.0536
Epoch 5/20
 - 6s - loss: 4.3653 - acc: 0.0555
Epoch 6/20
 - 6s - loss: 4.3477 - acc: 0.0657
Epoch 7/20
Epoch 7/20 - 6s - loss: 4.3300 - acc: 0.0651
Epoch 8/20
 - 6s - loss: 4.3253 - acc: 0.0651
Epoch 9/20

 - 6s - loss: 4.3220 - acc: 0.0710
Epoch 10/20
Epoch 10/20 - 6s - loss: 4.3091 - acc: 0.0779
Epoch 11/20
 - 6s - loss: 4.3073 - acc: 0.0762
Epoch 12/20
 - 6s - loss: 4.3019 - acc: 0.0766
Epoch 12/20
Epoch 13/20
Epoch 13/20
 - 6s - loss: 4.2974 - acc: 0.0799
Epoch 14/20
 - 6s - loss: 4.2983 - acc: 0.0808
Epoch 15/20
 - 6s - loss: 4.2956 - acc: 0.0812
Epoch 16/20
 - 6s - loss: 4.2889 - acc: 0.0848
Epoch 17/20
 - 6s - loss: 4.2866 - acc: 0.0868
Epoch 18/20
 - 6s - loss: 4.2838 - acc: 0.0812
Epoch 19/20
 - 6s - loss: 4.2816 - acc: 0.0871


## Few-shot

In [28]:
episode_generator = fewshot_dataset.get_fewshot_generator(n_way=n_way, 
                                                          k_shot=k_shot, 
                                                          query_size=query_size,
                                                          support_generator_args=support_generator_args)

Train few-shot model:

In [29]:
fewshot_optimizer = tf.keras.optimizers.Adam(lr=1e-3) 

In [30]:
fewshot_model = BaselineFewShotModel(backbone, n_way)
accuracies = baseline_fewshot_test(model=fewshot_model,
                                   generator=episode_generator, 
                                   optimizer=fewshot_optimizer,
                                   batch_size=fewshot_batch_size,
                                   support_epochs=support_epochs,
                                   n_episodes=n_episodes,
                                   model_name='baseline-fewshot',
                                   tensorboard=True,
                                   log_dir=log_dir,
                                   period=10)

  0%|          | 0/10 [00:00<?, ?it/s]/mnt/disk/venv/dpy3/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
Average acc: 42.00%: 100%|██████████| 10/10 [00:53<00:00,  5.45s/it]


## Report

Calculate 95% confidence interval:

In [25]:
acc_int = stats.t.interval(0.95, len(accuracies) - 1, loc=np.mean(accuracies), scale=stats.sem(accuracies))
print("5-shot test metric: {}% +- {}%".format(round(np.mean(accuracies) * 100, 2),
                                              round((np.mean(acc_int) - acc_int[0]) * 100, 2)))

5-shot test metric: 46.0% +- 6.49%
